# 🍓 Strawberry Vision - Google Colab Eğitim Pipeline

Bu notebook, Roboflow'dan dataset indirip YOLOv8 ile çilek tespit ve olgunluk sınıflandırma modeli eğitmenizi sağlar.

## 📋 Pipeline Adımları
1. **Google Drive Bağlantısı** - Checkpoint kaydetme için
2. **Kurulum** - Bağımlılıklar ve proje yapısı
3. **Roboflow Dataset** - API ile otomatik indirme
4. **Sınıf Etiketleme** - Otomatik standardizasyon
5. **Model Eğitimi** - YOLOv8 eğitimi (her 10 epoch'ta kayıt)
6. **Inference** - Eğitilmiş model ile tahmin
7. **Sonuçları Kaydet** - Model ve raporları Drive'a kaydet

## ⚡ Hızlı Başlangıç
1. Runtime > Change runtime type > **GPU** seçin
2. Tüm hücreleri sırayla çalıştırın
3. Google Drive'a bağlanın (Bölüm 1)
4. Roboflow API key'inizi girin (Bölüm 3)

---

## 💾 Bölüm 1: Google Drive Bağlantısı

Eğitim sırasında checkpoint'leri Google Drive'a kaydedeceğiz.

In [ ]:
# 1.1. Google Drive'ı Bağla
from google.colab import drive
import os
from pathlib import Path

print("📁 Google Drive bağlanıyor...")
drive.mount('/content/drive')

# Proje klasörü oluştur
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/StrawberryVision'
Path(DRIVE_PROJECT_PATH).mkdir(parents=True, exist_ok=True)

# Checkpoint klasörü
CHECKPOINT_DIR = f"{DRIVE_PROJECT_PATH}/checkpoints"
Path(CHECKPOINT_DIR).mkdir(parents=True, exist_ok=True)

print(f"✅ Drive bağlandı: {DRIVE_PROJECT_PATH}")
print(f"📦 Checkpoint klasörü: {CHECKPOINT_DIR}")

## 📦 Bölüm 2: Kurulum ve Proje Yapısı

In [ ]:
# 2.1. Bağımlılıkları Yükle
print("📦 Bağımlılıklar yükleniyor...")

!pip install -q ultralytics>=8.0.0
!pip install -q opencv-python>=4.8.0
!pip install -q numpy>=1.24.0
!pip install -q matplotlib>=3.7.0
!pip install -q roboflow>=1.1.0
!pip install -q pyyaml>=6.0

print("✅ Kurulum tamamlandı!")

In [ ]:
# 2.2. Proje Klasörlerini Oluştur
folders = ['datasets', 'runs', 'models']
for folder in folders:
    Path(folder).mkdir(exist_ok=True)
    print(f"✅ {folder}/")

print("\n📁 Proje yapısı hazır!")

## 🌐 Bölüm 3: Roboflow Dataset İndirme

### Önerilen Datasetler:
1. **Strawberry Ripeness Classification** ⭐ - `fruit-detection/strawberry-ripeness`
2. **Strawberry Detection Dataset** - `strawberry-detection/strawberry-detection-dataset`
3. **Agricultural Strawberry** - `agriculture/strawberry-field-detection`

### API Key Alma:
https://app.roboflow.com/settings/api

In [ ]:
# 3.1. Roboflow API Ayarları
from roboflow import Roboflow

# ⚠️ API KEY'İNİZİ BURAYA GİRİN
API_KEY = "YOUR_API_KEY_HERE"  # https://app.roboflow.com/settings/api

# Dataset seçimi (önerilen)
WORKSPACE = "fruit-detection"
PROJECT = "strawberry-ripeness"
VERSION = 1

print(f"📥 Dataset: {WORKSPACE}/{PROJECT} v{VERSION}")

In [ ]:
# 3.2. Dataset İndir
if API_KEY == "YOUR_API_KEY_HERE":
    print("⚠️  Lütfen API KEY'inizi girin!")
    print("API Key almak için: https://app.roboflow.com/settings/api")
else:
    print("🔄 Roboflow'dan indiriliyor...")
    
    rf = Roboflow(api_key=API_KEY)
    project = rf.workspace(WORKSPACE).project(PROJECT)
    dataset = project.version(VERSION).download("yolov8", location="datasets/roboflow")
    
    print("✅ Dataset indirildi: datasets/roboflow")
    print(f"📊 Dataset bilgileri: {dataset.location}")

## 🔄 Bölüm 4: Sınıf Etiketlerini Standardize Et

Roboflow'dan gelen sınıf isimleri farklı olabilir. Standart formata çeviriyoruz:
- `strawberry_ripe` - Olgun çilek
- `strawberry_semi_ripe` - Yarı olgun çilek  
- `strawberry_unripe` - Olgun olmayan çilek

In [ ]:
# 4.1. Sınıf Mapping ve Label Güncelleme
import yaml
import shutil

CLASS_MAPPING = {
    "ripe": "strawberry_ripe",
    "semi-ripe": "strawberry_semi_ripe",
    "semi_ripe": "strawberry_semi_ripe",
    "unripe": "strawberry_unripe",
    "green": "strawberry_unripe",
    "strawberry": "strawberry_ripe",
}

STANDARD_CLASSES = {
    0: "strawberry_ripe",
    1: "strawberry_semi_ripe",
    2: "strawberry_unripe",
}

yaml_path = "datasets/roboflow/data.yaml"

if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    old_names = data_config.get('names', [])
    print(f"Eski sınıflar: {old_names}")
    
    # ID mapping oluştur
    if isinstance(old_names, list):
        old_names = {i: name for i, name in enumerate(old_names)}
    
    id_mapping = {}
    for old_id, old_name in old_names.items():
        mapped_name = CLASS_MAPPING.get(old_name.lower(), "strawberry_ripe")
        for new_id, new_name in STANDARD_CLASSES.items():
            if mapped_name == new_name:
                id_mapping[old_id] = new_id
                break
    
    # Label dosyalarını güncelle
    def update_labels(labels_dir, id_mapping):
        updated_count = 0
        for label_file in Path(labels_dir).glob('*.txt'):
            with open(label_file, 'r') as f:
                lines = f.readlines()
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    old_class_id = int(parts[0])
                    new_class_id = id_mapping.get(old_class_id, old_class_id)
                    parts[0] = str(new_class_id)
                    new_lines.append(' '.join(parts) + '\n')
                    if new_class_id != old_class_id:
                        updated_count += 1
            with open(label_file, 'w') as f:
                f.writelines(new_lines)
        return updated_count
    
    total_updated = 0
    for split in ['train', 'valid', 'test']:
        labels_dir = f"datasets/roboflow/{split}/labels"
        if os.path.exists(labels_dir):
            count = update_labels(labels_dir, id_mapping)
            total_updated += count
            print(f"✅ {split}: {count} etiket güncellendi")
    
    # Yeni data.yaml oluştur
    new_data_yaml = {
        'path': os.path.abspath('datasets/roboflow'),
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'nc': 3,
        'names': STANDARD_CLASSES
    }
    
    with open('datasets/roboflow/data.yaml', 'w') as f:
        yaml.dump(new_data_yaml, f, default_flow_style=False)
    
    print(f"\n✅ Toplam {total_updated} etiket güncellendi")
    print(f"✅ Standart sınıflar: {STANDARD_CLASSES}")
else:
    print("⚠️  data.yaml bulunamadı")

## 🎓 Bölüm 5: YOLOv8 Model Eğitimi (Her 10 Epoch'ta Kayıt)

### Önerilen Parametreler:
- **Hızlı Test**: yolov8n.pt, 20 epoch, batch 16
- **Orta Kalite**: yolov8s.pt, 50 epoch, batch 16  
- **Yüksek Kalite**: yolov8m.pt, 100 epoch, batch 8-16

In [ ]:
# 5.1. Eğitim Parametreleri
TRAIN_CONFIG = {
    'model': 'yolov8n.pt',
    'epochs': 50,
    'batch': 16,
    'imgsz': 640,
    'device': 0,
    'project': 'runs/train',
    'name': 'strawberry_exp',
    'patience': 50,
    'save_period': 10,        # ⭐ Her 10 epoch'ta kaydet
    'optimizer': 'AdamW',
    'lr0': 0.01,
    'lrf': 0.01,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 10.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.1,
}

print("🎯 Eğitim Parametreleri:")
for key, value in TRAIN_CONFIG.items():
    print(f"  {key}: {value}")

print(f"\n💾 Checkpoint kayıt: Her {TRAIN_CONFIG['save_period']} epoch'ta")
print(f"📁 Drive klasörü: {CHECKPOINT_DIR}")

In [ ]:
# 5.2. Model Eğitimi Başlat
from ultralytics import YOLO
import time

print("🚀 Model eğitimi başlıyor...")
print("⏱️  Bu işlem 30-60 dakika sürebilir (GPU ile)")
print(f"💾 Her {TRAIN_CONFIG['save_period']} epoch'ta checkpoint Drive'a kaydedilecek\n")

# Model yükle
model = YOLO(TRAIN_CONFIG['model'])

# Eğitim başlat
start_time = time.time()
results = model.train(
    data='datasets/roboflow/data.yaml',
    **TRAIN_CONFIG
)

training_time = time.time() - start_time

print(f"\n✅ Eğitim tamamlandı! Süre: {training_time/60:.1f} dakika")
print(f"📊 Sonuçlar: {TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}")
print(f"🏆 En iyi model: {TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}/weights/best.pt")

In [ ]:
# 5.3. Checkpoint'leri Google Drive'a Kopyala
results_dir = f"{TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}"
weights_dir = f"{results_dir}/weights"

if os.path.exists(weights_dir):
    print("💾 Checkpoint'ler Drive'a kopyalanıyor...\n")
    
    # Weights klasörünü kopyala
    drive_weights_dir = f"{CHECKPOINT_DIR}/{TRAIN_CONFIG['name']}_weights"
    if os.path.exists(drive_weights_dir):
        shutil.rmtree(drive_weights_dir)
    shutil.copytree(weights_dir, drive_weights_dir)
    
    # Checkpoint dosyalarını listele
    checkpoint_files = list(Path(drive_weights_dir).glob('*.pt'))
    print(f"✅ {len(checkpoint_files)} checkpoint Drive'a kaydedildi:")
    for ckpt in sorted(checkpoint_files):
        size_mb = ckpt.stat().st_size / (1024*1024)
        print(f"  📦 {ckpt.name} ({size_mb:.1f} MB)")
    
    print(f"\n📁 Drive konumu: {drive_weights_dir}")
else:
    print("⚠️  Weights klasörü bulunamadı")

In [ ]:
# 5.4. Eğitim Sonuçlarını Görselleştir
from IPython.display import Image, display
import matplotlib.pyplot as plt

print("📊 Eğitim Grafikleri:\n")

# Results
if os.path.exists(f"{results_dir}/results.png"):
    display(Image(filename=f"{results_dir}/results.png"))

# Confusion Matrix
if os.path.exists(f"{results_dir}/confusion_matrix.png"):
    display(Image(filename=f"{results_dir}/confusion_matrix.png"))

# Validation Predictions
if os.path.exists(f"{results_dir}/val_batch0_pred.jpg"):
    display(Image(filename=f"{results_dir}/val_batch0_pred.jpg"))

## 🔍 Bölüm 6: Model Değerlendirme ve Inference

In [ ]:
# 6.1. Model Değerlendirme
best_model_path = f"{results_dir}/weights/best.pt"

if os.path.exists(best_model_path):
    print("📊 Model değerlendiriliyor...\n")
    
    model = YOLO(best_model_path)
    metrics = model.val(data='datasets/roboflow/data.yaml')
    
    print("\n✅ Değerlendirme Sonuçları:")
    print(f"  mAP@0.5: {metrics.box.map50:.4f}")
    print(f"  mAP@0.5:0.95: {metrics.box.map:.4f}")
    print(f"  Precision: {metrics.box.mp:.4f}")
    print(f"  Recall: {metrics.box.mr:.4f}")
else:
    print("⚠️  Model bulunamadı")

In [ ]:
# 6.2. Test Görüntüleri ile Inference
import cv2
import numpy as np

test_images_dir = "datasets/roboflow/valid/images"
test_images = list(Path(test_images_dir).glob('*.jpg'))[:5]

if test_images and os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    
    for img_path in test_images:
        results = model(str(img_path))
        annotated = results[0].plot()
        
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Tespit: {img_path.name}")
        plt.show()
        
        boxes = results[0].boxes
        print(f"\n{img_path.name}: {len(boxes)} çilek tespit edildi")
        
        if len(boxes) > 0:
            class_counts = {}
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = STANDARD_CLASSES[cls_id]
                class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
            print(f"Dağılım: {class_counts}")
        print("-" * 50)
else:
    print("⚠️  Test görüntüleri veya model bulunamadı")

## 💾 Bölüm 7: Sonuçları Drive'a Kaydet

Tüm eğitim sonuçları ve modeller Google Drive'a kaydedilecek.

In [ ]:
# 7.1. Tüm Sonuçları Drive'a Kopyala
if os.path.exists(results_dir):
    print("📦 Tüm sonuçlar Drive'a kopyalanıyor...\n")
    
    drive_results_dir = f"{DRIVE_PROJECT_PATH}/results/{TRAIN_CONFIG['name']}"
    if os.path.exists(drive_results_dir):
        shutil.rmtree(drive_results_dir)
    shutil.copytree(results_dir, drive_results_dir)
    
    # Boyut hesapla
    total_size = sum(f.stat().st_size for f in Path(drive_results_dir).rglob('*') if f.is_file())
    
    print(f"✅ Sonuçlar Drive'a kaydedildi")
    print(f"📁 Konum: {drive_results_dir}")
    print(f"📊 Toplam boyut: {total_size / (1024*1024):.1f} MB")
    
    print("\n📋 Kaydedilen dosyalar:")
    print("  ✅ Eğitim grafikleri (results.png, confusion_matrix.png)")
    print("  ✅ Model checkpoint'leri (weights/)")
    print("  ✅ Validation tahminleri")
    print("  ✅ Eğitim logları")
else:
    print("⚠️  Sonuç klasörü bulunamadı")

In [ ]:
# 7.2. En İyi Modeli Ayrıca Kaydet
if os.path.exists(best_model_path):
    best_model_drive = f"{DRIVE_PROJECT_PATH}/best_models/strawberry_best_{TRAIN_CONFIG['name']}.pt"
    Path(best_model_drive).parent.mkdir(parents=True, exist_ok=True)
    
    shutil.copy(best_model_path, best_model_drive)
    
    size_mb = Path(best_model_drive).stat().st_size / (1024*1024)
    print(f"🏆 En iyi model Drive'a kaydedildi")
    print(f"📁 Konum: {best_model_drive}")
    print(f"📊 Boyut: {size_mb:.1f} MB")
else:
    print("⚠️  Model bulunamadı")

## 📝 Özet ve Notlar

### ✅ Tamamlanan Adımlar
1. ✅ Google Drive bağlandı
2. ✅ Roboflow'dan dataset indirildi
3. ✅ Sınıf etiketleri standardize edildi
4. ✅ YOLOv8 modeli eğitildi
5. ✅ Her 10 epoch'ta checkpoint kaydedildi
6. ✅ Model değerlendirildi
7. ✅ Sonuçlar Drive'a kaydedildi

### 💾 Drive Klasör Yapısı
```
MyDrive/StrawberryVision/
├── checkpoints/
│   └── strawberry_exp_weights/
│       ├── epoch10.pt
│       ├── epoch20.pt
│       ├── epoch30.pt
│       ├── best.pt
│       └── last.pt
├── results/
│   └── strawberry_exp/
│       ├── weights/
│       ├── results.png
│       └── confusion_matrix.png
└── best_models/
    └── strawberry_best_strawberry_exp.pt
```

### 📊 Performans Hedefleri
- mAP@0.5 ≥ %80
- Precision ≥ %85
- Recall ≥ %75

### 🔧 Sorun Giderme
- **Out of Memory**: `batch` değerini düşürün (16 → 8)
- **Yavaş Eğitim**: GPU'nun aktif olduğunu kontrol edin
- **Drive Bağlantı Hatası**: Hücreyi tekrar çalıştırın

### 📚 Kaynaklar
- [YOLOv8 Docs](https://docs.ultralytics.com)
- [Roboflow Universe](https://universe.roboflow.com)

---

**🎉 Tebrikler!** Modeliniz eğitildi ve Google Drive'a kaydedildi. Artık kendi görüntülerinizde kullanabilirsiniz.